In [1]:
import pandas as pd
from datetime import datetime, date

In [2]:
def make_df(df, year=[2021],bins=[20,150],
            x='Category', quantile=0.1, topcut=20,
            aggregate_by='average'):
    # load/prepare datafile
    
    df = df.loc[df['Date'].dt.year.isin(year)]
    df = df.set_index('Date')

    # choose x, bins
    if bins:
        if bins == 'Time Comparison':
            grabs = df.resample('M').count()
            most_recent = grabs.iloc[-int(grabs.shape[0]/2):].index
            previous = grabs.loc[~grabs.index.isin(most_recent)].index
            labels = [f'before {previous[-1].date()}', f'after {previous[-1].date()}']
            
            if x != 'Date':
                dff = df.groupby(x).resample('M')[['Amount']].mean()
            else:
                dff = df.resample('M')[['Amount']].sum()
            dff = dff.reset_index()
            dff.loc[dff.Date.isin(most_recent), 'Bin'] = labels[1]
            dff.loc[dff.Date.isin(previous), 'Bin'] = labels[0]
            
        else:
            labels = [f'less than {bins[0]}', f'between {bins[0]} and {bins[1]}', f'greater than {bins[1]}']
            if x != 'Date':
                dff = df.loc[(df.Amount < bins[0])].groupby(x).resample('M')[['Amount']].sum()
                dff.columns = [labels[0]]
                dff[labels[1]] = df.loc[(df.Amount >= bins[0]) &
                                        (df.Amount < bins[1])].groupby(x).resample('M')[['Amount']].sum()
                dff[labels[2]] = df.loc[(df.Amount > bins[1])].groupby(x).resample('M')[['Amount']].sum()
                dff = dff.reset_index()
                dff = dff.melt(id_vars=[x, 'Date'], value_name='Amount', var_name='Bin')
            else:
                dff = df.loc[(df.Amount < bins[0])].resample('M')[['Amount']].sum()
                dff.columns = [labels[0]]
                dff[labels[1]] = df.loc[(df.Amount >= bins[0]) &
                                        (df.Amount < bins[1])].resample('M')[['Amount']].sum()
                dff[labels[2]] = df.loc[(df.Amount > bins[1])].resample('M')[['Amount']].sum()
                dff = dff.reset_index()
                dff = dff.melt(id_vars=['Date'], value_name='Amount', var_name='Bin')
    else:
        
        if x != 'Date':
            dff = df.groupby(x).resample('M')[['Amount']].sum()
            dff = dff.reset_index()
        else:
            dff = df.resample('M')[['Amount']].sum()
            dff = dff.reset_index()
            
    if quantile:
        mymap1 = (dff.resample(on='Date', rule='M').sum()['Amount'] <= dff.resample(on='Date', rule='M').sum().quantile(quantile)[0]).to_dict()
        mymap2 = (dff.resample(on='Date', rule='M').sum()['Amount'] >= dff.resample(on='Date', rule='M').sum().quantile(1-quantile)[0]).to_dict()

        mymap = {}
        for key, val in mymap1.items():
            if val == True:
                mymap[key] = 'Bottom'
            elif mymap2[key] == True:
                mymap[key] = 'Top'
            else:
                mymap[key] = 'Middle'
        dff['Quantile'] = dff['Date'].map(mymap)

    # cut off small - value categories (x)
    if x != 'Date':
        dfftop = dff.groupby(x)[['Amount']].sum().sort_values('Amount', ascending=False)[:topcut]
        dfftop = dfftop.reset_index()
        dff1 = dff.loc[dff[x].isin(dfftop[x])]
        dff2 = dff.loc[~dff[x].isin(dfftop[x])]
        if bins and quantile:
            dffbot = pd.DataFrame(dff2.groupby(['Date', 'Bin', 'Quantile'])['Amount'].sum()).reset_index()
        elif quantile:
            dffbot = pd.DataFrame(dff2.groupby(['Date', 'Quantile'])['Amount'].sum()).reset_index()
        elif bins:
            dffbot = pd.DataFrame(dff2.groupby(['Date', 'Bin'])['Amount'].sum()).reset_index()
        else:
            dffbot = pd.DataFrame(dff2.groupby(['Date'])['Amount'].sum()).reset_index()
        dffbot[x] = 'Other'
        dff = pd.concat([dff1, dffbot])

    # collapse onto groupby
    if aggregate_by == 'average':
        if bins and quantile:
            dff = dff.groupby([x, 'Quantile', 'Bin'])[['Amount']].mean().reset_index()
        elif quantile:
            dff = dff.groupby([x, 'Quantile'])[['Amount']].mean().reset_index()
        elif bins:
            dff = dff.groupby([x, 'Bin'])[['Amount']].mean().reset_index()
        else:
            dff = dff.groupby([x])[['Amount']].mean().reset_index()
    else:
        if bins and quantile:
            dff = dff.groupby([x, 'Quantile', 'Bin'])[['Amount']].sum().reset_index()
        elif quantile:
            dff = dff.groupby([x, 'Quantile'])[['Amount']].sum().reset_index()
        elif bins:
            dff = dff.groupby([x, 'Bin'])[['Amount']].sum().reset_index()
        else:
            dff = dff.groupby([x])[['Amount']].sum().reset_index()

    # sort results, up to 3 sort categories
    if x != 'Date':
        sorter1 = dff.groupby(x)['Amount'].sum().sort_values(ascending=False).index
        sorterIndex1 = dict(zip(sorter1, range(len(sorter1))))
    if quantile:
        sorter2 = ['Top', 'Middle', 'Bottom']
        sorterIndex2 = dict(zip(sorter2, range(len(sorter2))))
    if bins:
        sorter3 = labels
        sorterIndex3 = dict(zip(sorter3, range(len(sorter3))))
        
    if bins and x != 'Date' and quantile:
        sorters = [sorterIndex1, sorterIndex2, sorterIndex3]
    elif bins and quantile:
        sorters = [sorterIndex2, sorterIndex3]
    elif quantile:
        sorters = [sorterIndex2]
    elif bins:
        sorters = [sorterIndex3]
        
    if x != 'Date':
        dff['sort1'] = dff[x].map(sorterIndex1)
    if quantile:
        dff['sort2'] = dff.Quantile.map(sorterIndex2)
    if bins:
        dff['sort3'] = dff.Bin.map(sorterIndex3)
        if x != 'Date' and quantile:
            dff = dff.sort_values(['sort1', 'sort2', 'sort3']).reset_index(drop=True)
            dff = dff.drop(['sort1', 'sort2', 'sort3'], axis=1)
        elif quantile:
            dff = dff.sort_values(['sort2', 'sort3']).reset_index(drop=True)
            dff = dff.drop(['sort2', 'sort3'], axis=1)
        elif x != 'Date':
            dff = dff.sort_values(['sort1', 'sort3']).reset_index(drop=True)
            dff = dff.drop(['sort1', 'sort3'], axis=1)
        else:
            dff = dff.sort_values(['sort3']).reset_index(drop=True)
            dff = dff.drop(['sort3'], axis=1)
    else:
        if x != 'Date' and quantile:
            dff = dff.sort_values(['sort1', 'sort2']).reset_index(drop=True)
            dff = dff.drop(['sort1', 'sort2'], axis=1)
            sorters = [sorterIndex1, sorterIndex2]
            labels = None
        elif quantile:
            dff = dff.sort_values(['sort2']).reset_index(drop=True)
            dff = dff.drop(['sort2'], axis=1)
            sorters = [sorterIndex2]
            labels = None
        elif x != 'Date':
            dff = dff.sort_values(['sort1']).reset_index(drop=True)
            dff = dff.drop(['sort1'], axis=1)
            sorters = [sorterIndex1]
            labels = None
        else:
            sorters = None
            labels = None
    return dff, sorters, labels

In [3]:
df = pd.read_csv('../data/transactions.csv', parse_dates=['Date'])

# df = process_data(df, pivot_dates=False)

categories = list(df.Category.unique())
income = ['Income', 'Paycheck', 'Transfer', 'Federal Tax', 'Taxes', 'Rental Income', 'Interest Income']
taxes = ['Federal Tax', 'Taxes']
internal_acc = ['Credit Card Payment', 'Transfer', 'Financial']
non_expense =  income + taxes + internal_acc
expenses = [i for i in categories if i not in non_expense]

df = df.loc[df['Category'].isin(expenses)]
df = df.loc[df['Amount'] < 8000]

# Primary Chart Data

In [4]:
dff, _, _ = make_df(df, x='Date', year=[2021], quantile=0.1, bins=None)
monthly = dff.set_index("Date")
monthly.columns = ['Decile', 'Amount']


for quantile, label in zip([.25,.5], ['Quartile', 'Median']):
    dff, _, _ = make_df(df, x='Date', year=[2021], quantile=quantile, bins=None)
    dff = dff.set_index("Date")
    monthly[label] = dff['Quantile']

monthly.index = monthly.index.month
monthly = monthly.reset_index()
monthly = monthly.sort_values('Date')

monthly = monthly.reset_index(drop=True)
monthly = monthly[['Date', 'Amount', 'Decile', 'Quartile', 'Median']]
monthly['Amount'] = round(monthly['Amount'])
# monthly.to_json('static_data/monthly_spending.json')

In [18]:
dff, _, _ = make_df(df, x='Date', year=[2021], quantile=None, bins=[20, 150])
dff = round(dff)
monthly = dff.groupby(['Date', 'Bin'], sort=False)['Amount'].sum().unstack('Bin').reset_index()#.to_json('static_data/category_spending_10.json')
monthly = monthly.set_index('Date')
monthly.index = monthly.index.month
monthly = monthly.reset_index()
monthly = monthly.sort_values('Date')

monthly = monthly.reset_index(drop=True)

In [20]:
monthly.to_json('static_data/monthly_spending_threshold.json')

In [24]:
dff, _, _ = make_df(df, x='Date', year=[2021], quantile=None, bins='Time Comparison')
dff = round(dff)
monthly = dff.groupby(['Date', 'Bin'], sort=False)['Amount'].sum().unstack('Bin').reset_index()#.to_json('static_data/category_spending_10.json')
monthly = monthly.set_index('Date')
monthly.index = monthly.index.month
monthly = monthly.reset_index()
monthly = monthly.sort_values('Date')

monthly = monthly.reset_index(drop=True)

In [26]:
monthly.to_json('static_data/monthly_spending_time.json')

# Secondary Chart Data

In [202]:
dff, _, _ = make_df(df, x='Category', year=[2021], quantile=0.1, bins=None)
dff = round(dff)
dff.groupby(['Category', 'Quantile'], sort=False)['Amount'].sum().unstack('Quantile').reset_index().to_json('static_data/category_spending_10.json')
df1 = pd.concat({"Top/Bottom 10%": dff}, axis=1, names=["Quantile", "Data"])

dff, _, _ = make_df(df, x='Category', year=[2021], quantile=0.25, bins=None)
dff = round(dff)
dff.groupby(['Category', 'Quantile'], sort=False)['Amount'].sum().unstack('Quantile').reset_index().to_json('static_data/category_spending_25.json')
df2 = pd.concat({"Top/Bottom Quartile": dff}, axis=1, names=["Quantile", "Data"])

dff, _, _ = make_df(df, x='Category', year=[2021], quantile=0.5, bins=None)
dff = round(dff)
dff.groupby(['Category', 'Quantile'], sort=False)['Amount'].sum().unstack('Quantile').reset_index().to_json('static_data/category_spending_50.json')
df3 = pd.concat({"Above/Bellow Median": dff}, axis=1, names=["Quantile", "Data"])

dff, _, _ = make_df(df, x='Category', year=[2021], quantile=None, bins=None)
dff = round(dff)
dff.to_json('static_data/category_spending_none.json')
df4 = pd.concat({"None": dff}, axis=1, names=["Quantile", "Data"])

In [23]:
dff, _, _ = make_df(df, x='Category', year=[2021], quantile=None, bins=[20, 150])
dff = round(dff)
dff = dff.groupby(['Category', 'Bin'], sort=False)['Amount'].sum().unstack('Bin').reset_index()
dff.to_json('static_data/category_spending_threshold.json')

Bin,Category,less than 20,between 20 and 150,greater than 150
0,Groceries,57.0,254.0,259.0
1,Gym,4.0,56.0,420.0
2,Travel,5.0,21.0,318.0
3,Rental Car & Taxi,23.0,146.0,155.0
4,Shopping,16.0,141.0,121.0


In [28]:
dff, _, _ = make_df(df, x='Category', year=[2021], quantile=None, bins='Time Comparison')
dff = round(dff)
dff = dff.groupby(['Category', 'Bin'], sort=False)['Amount'].sum().unstack('Bin').reset_index()
dff
# dff.to_json('static_data/category_spending_threshold.json')

Bin,Category,before 2021-06-30,after 2021-06-30
0,Hotel,1858.0,505.0
1,Mortgage & Rent,1095.0,1056.0
2,Student Loan,1000.0,1000.0
3,Other,185.0,297.0
4,Furnishings,408.0,38.0
5,Air Travel,NaN,422.0
6,Vacation,203.0,123.0
7,Service & Parts,200.0,115.0
8,Home Services,298.0,NaN
9,Sporting Goods,114.0,154.0
